In [99]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import time
import json

In [2]:
# limited data IN
df = pd.read_csv("../outputs/geoc_inv_US_2011_2020.csv", sep=";")

In [9]:
# create locations table
locations = df[["lat", "lng"]].drop_duplicates()
locations["location_id"] = locations.index + 1

In [108]:
# add Point geometry for spatial join
locations["geometry"] = [Point(xy) for xy in zip(locations.lng, locations.lat)]
locations = gpd.GeoDataFrame(locations, geometry="geometry", crs="EPSG:4326")

In [116]:
# census tract geoms
tract_geoms = gpd.GeoDataFrame.from_features(
    [json.loads(e.strip('\n')) for e in open('../data/shape_files/censustract_geoms_top50.geojson').readlines()]
)
tract_geoms = tract_geoms.set_crs("epsg:4326")

In [117]:
def locations_to_census_tracts(points, tracts):
    """spatial join locations to census tracts"""
    location_with_tract = gpd.sjoin(
        points,
        tracts,
        "left",
        "within",
    )
    return location_with_tract

In [120]:
# add census tract IDs to locations
locations = locations_to_census_tracts(locations, tract_geoms)
locations.dropna(subset=["full_geoid"], inplace=True)

In [11]:
# join location IDs to df
df2 = pd.merge(
    df,
    locations,
    on=["lat", "lng"],
    how="left"
)

In [64]:
def edgelist_construction(df, key_cols, directed):
    """create location-location edgelist"""
    
    # focus the dataframe
    df = df[key_cols].drop_duplicates()


    # create edgelist by join
    el = pd.merge(
        df,
        df,
        on=key_cols[0],
        suffixes=["1", "2"]
    )

    # directed
    if directed == True:
        el = el[el.iloc[:, 1] != el.iloc[:, 2]]
    else:
        el = el[el.iloc[:, 1] < el.iloc[:, 2]]
    
    # final dataframe
    el = el.iloc[:, 1:]

    return el

In [71]:
# full combination from MultiIndex product -- should be fast!
start_time = time.time()
el = edgelist_construction(df2, key_cols=["appln_id", "location_id"], directed=False)
print("--- %s seconds ---" % round((time.time() - start_time), 3))

--- 0.237 seconds ---


In [73]:
# add the coordinates to the edgelist
locations

,lat,lng,location_id
0,42.2529,-73.7910,1
1,32.8629,-96.9584,2
2,42.0521,-88.0135,3
5,37.3207,-122.0550,6
6,37.4042,-122.1260,7
...,...,...,...
1330748,38.2029,-75.6924,1330749
1330979,37.6751,-122.4910,1330980
1331954,40.0779,-75.6877,1331955
1332946,35.5479,-87.5520,1332947


In [ ]:
# add the 